HW4
1i

In [19]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

df = pd.read_csv('Final_Imputed_Whole.csv',index_col=0)
print(df)
X = df.drop(columns=['ViolentCrimesPerPop'])
y = df['ViolentCrimesPerPop']
y1 = y[0:1495]  # training
print(X)
print(y)
print(y1)

      population  householdsize  racepctblack  racePctWhite  racePctAsian  \
0           0.19           0.33          0.02          0.90          0.12   
1           0.00           0.16          0.12          0.74          0.45   
2           0.00           0.42          0.49          0.56          0.17   
3           0.04           0.77          1.00          0.08          0.12   
4           0.01           0.55          0.02          0.95          0.09   
...          ...            ...           ...           ...           ...   
1989        0.01           0.40          0.10          0.87          0.12   
1990        0.05           0.96          0.46          0.28          0.83   
1991        0.16           0.37          0.25          0.69          0.04   
1992        0.08           0.51          0.06          0.87          0.22   
1993        0.20           0.78          0.14          0.46          0.24   

      racePctHisp  agePct12t21  agePct12t29  agePct16t24  agePct65up  ...  

For PCR, PCA gives axes and then Linear Regression gives us the fit

As performing PCR requires first performing PCA, a question does arise:
When should PCA be performed?

We'll answer this question, similarly to how we answered similar previous questions in previous portions of the assignment.

Thus PCA will be performed on the whole dataset. Thus PCA will be performed on the train+validation+test dataset at once, from which various portions will be used for various purposes.

We have a few ways to cross validate over each selected number of components then, to select an ideal M. They are :

1. Choose no. of components first, then perform CV
2. Choose a CV iteration then iterate over all components
3. Same as 1. but saves, over each CV iteration, train and test fold division indices

Method 1. is the simplest, however has one issue : As CV is a random in-built method, over different values of number of components, we won't have 'sample correspondence' between CV iterations. That is, while for each selected value of no. of principal components, the folds selected will stay constant, obviously, for different no. of components, the folds will differ.

Ex. For m=2, m=3 the five folds (in 5-fold CV) may differ. Obviously over each of the five iterations for 5-fold CV over each individual m, say m=2, the folds will stay same over the iterations due to the nature of inbuilt function

Method 2. Obviously solves this as the fold indices are obtained first. Due to the requirement of PCA having to be performed on the entire set, training, validation, and test, this surely becomes complicated.

Method 3. Same as 1 but by saving CV-indices over each iteration, over different m the folds stay constant.

We'll go with Method 1, as it is the simplest, and we assume that results for different m can be compared over different folds in so far as, obviously, folds within each m stays constant. As, obviously, the number of folds become larger, this becomes true, and for leave one out cross validation all the three methods are exactly equal.

We'll select total folds = 10 here, owing to this reason, and use Method 1

Range of m will be {1,2,3,...........,122} 
Ideally it should be 121, since PCR's purpose is to reduce dimensions, but we'll check results for m=122 also

In [20]:
m_arr = np.arange(0,122)
mse_sum_comp = np.zeros(122)
i=0
for i in m_arr:
    selected_components = i+1
    pca = PCA(n_components=selected_components)
    pca_fit = pca.fit(X) # X is whole feature set, or the entire available dataset
    X_whole_interim = pca_fit.transform(X) # Transformed feature, for the entire available dataset, Note sample correspondence 
                                           #  is always maintained
    X_whole = pd.DataFrame(X_whole_interim)# column headers generic 
    
    X1 = X_whole.iloc[0:1495,:]            # train on which CV performed, final test not needed right now, output corresponds to 
                                           # constant y1, iteration independent
    
    
    kf = KFold(n_splits=10,shuffle=True)
    
    for train_index, test_index in kf.split(X1):
        X1_train, X1_test = X1.iloc[train_index,:], X1.iloc[test_index,:]
        y1_train, y1_test = y1[train_index], y1[test_index]
        reg = LinearRegression()
        reg_fit = reg.fit(X1_train, y1_train)
        y_pred = reg_fit.predict(X1_test)
        mse = mean_squared_error(y1_test,y_pred)
        mse_sum_comp[i] = mse_sum_comp[i] + mse  # Calculates sum of mse's over 10 folds, places in relevant index


mse_comp = mse_sum_comp/10 # averaging the mse summed over 10 folds for all selected components

comp_selected_index = np.argmin(mse_comp)

comp_selected = comp_selected_index + 1

print('The number of components M selected via 10 fold CV is : \n',comp_selected)

# Refitting using those number of components

pca_final = PCA(n_components=comp_selected)
pca_final_fit = pca_final.fit(X)

X_final_whole_interim = pca_final_fit.transform(X)

X_final_whole = pd.DataFrame(X_final_whole_interim)

X_final_train = X_final_whole.iloc[0:1495,:]
X_final_test = X_final_whole.iloc[1495:1994,:]

y2 = y[1495:1994]  # y1 is train y, y2 is test y, y stay same throughout

final_reg = LinearRegression()
final_reg_fit = final_reg.fit(X_final_train,y1)

y_final_pred = final_reg_fit.predict(X_final_test)

mse_final = mean_squared_error(y2,y_final_pred)

print('The corresponding test MSE obtained on unseen data is : \n',mse_final)

print('Predicted unseen y test is : \n',y_final_pred)

print('True y test unseen is : \n',y2)
        
        
    

The number of components M selected via 10 fold CV is : 
 102
The corresponding test MSE obtained on unseen data is : 
 0.018019385649911124
Predicted unseen y test is : 
 [ 1.32719665e-01 -6.95554215e-02  2.14337724e-01  6.42060106e-02
  1.12247846e-01 -1.11446675e-03  2.50284869e-01  5.03581552e-01
  1.39569042e-01  3.36813286e-01  2.78381530e-01  5.95082104e-01
  2.27317759e-01  5.60654135e-01  4.27605226e-01  1.56778231e-01
  3.76914478e-01  5.08527727e-01  2.32174809e-01  3.82840852e-01
  1.56970690e-02  2.99089636e-02  2.13167893e-01  1.98663270e-01
  3.74455157e-02  4.21303817e-02  6.92698995e-02  5.06189922e-02
  3.93860014e-01  6.83439959e-01  1.38749453e-01  1.72861029e-01
  2.36930820e-01  2.28502155e-01  1.43734337e-01  9.61320397e-02
  3.50554918e-03  2.90537507e-01  2.06699878e-01  3.87179041e-01
  2.09361075e-01  7.26798412e-01  2.64537757e-01  4.71868979e-01
  4.18867175e-01  3.56061762e-01  3.67656350e-01  2.69176988e-01
  6.17851314e-01  5.08096990e-01  1.74440157e-01

Seeing in the output window, the,
Number of Principal Components, m, is obtained somewhere between 80-100
The value of test MSE obtained over this range is about 0.018
Again, as y is unchanged in all portions of the assignment, this figure of test MSE can be compared with other regressions done previously

m = somewhere between 80-100
test MSE = around 0.018

For PCR